In [160]:
import requests, json
import hashlib
import datetime
import pandas as pd 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, ArrayType, DateType, TimestampType
from pyspark.sql import functions as f
from pyspark.sql.functions import udf, from_unixtime, unix_timestamp, to_date
from datetime import timedelta, date
from delta.tables import DeltaTable

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 192, Finished, Available)

In [161]:
#Constants

const_timestamp = datetime.datetime.today().replace(second=0, microsecond=0)

api_url = r"https://v6.exchangerate-api.com/v6/479d2c7b7ab257e9cdf338e5/latest/USD"

currency_api_schema = StructType([StructField("currency_name", StringType(), True)\
                   ,StructField("currency_value", FloatType(), True)])

log_file_list = [
    "https://raw.githubusercontent.com/PacktPublishing/Data-Engineering-with-Apache-Spark-Delta-Lake-and-Lakehouse/main/project/prep/ecommerce_logs/electroniz_access_log_1.log",
    "https://raw.githubusercontent.com/PacktPublishing/Data-Engineering-with-Apache-Spark-Delta-Lake-and-Lakehouse/main/project/prep/ecommerce_logs/electroniz_access_log_2.log"
    ]

pd_dfs = [] 

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 193, Finished, Available)

**Create Delta Tables using SQL statements**

In [162]:
%%sql
CREATE TABLE IF NOT EXISTS silver_store_customers 
(customer_id Integer,
    customer_name String,
    address String,
    city String,
    postalcode String,
    country String,
    phone String,
    email String,
    credit_card String,
    updated_at Timestamp);

CREATE TABLE IF NOT EXISTS silver_store_orders
(order_number Integer,
    customer_id Integer,
    product_id Integer,
    order_date Date,
    units Integer,
    sale_price Float,
    currency String,
    order_mode String,
    sale_price_usd Float,
    updated_at Timestamp);

CREATE TABLE IF NOT EXISTS silver_products
(product_id Integer,
    product_name String,
    product_category String,
    updated_at Timestamp);

CREATE TABLE IF NOT EXISTS silver_currency
(currency_value Float,
    currency_name String,
    updated_at Timestamp);

CREATE TABLE IF NOT EXISTS silver_geolocation
(ip1 Integer,
    ip2 Integer,
    country_code String,
    country_name String,
    updated_at Timestamp);

CREATE TABLE IF NOT EXISTS silver_logs
(time String,
    remote_ip String,
    country_name String,
    ip_number Integer,
    request String,
    response String,
    agent String,
    updated_at Timestamp);

CREATE TABLE IF NOT EXISTS silver_esalesns
(customer_name String,
    address String,
    city String,
    country String,
    currency String,
    email String,
    order_date Date,
    order_mode String,
    order_number Integer,
    phone String,
    postalcode String,
    product_name String,
    sale_price Float,
    sale_price_usd Float,
    updated_at Timestamp);

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 200, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

**Create Delta Tables using API**

In [163]:
#Create CUSTOMERS silver Table
DeltaTable.createIfNotExists(spark) \
    .tableName('silver_store_customers') \
    .addColumn('customer_id', 'Integer') \
    .addColumn('customer_name', 'String') \
    .addColumn('address', 'String') \
    .addColumn('city', 'String') \
    .addColumn('postalcode', 'String') \
    .addColumn('country', 'String') \
    .addColumn('phone', 'String') \
    .addColumn('email', 'String') \
    .addColumn('credit_card', 'String') \
    .addColumn('updated_at', 'Timestamp') \
    .execute()


#Create ORDERS silver Table
DeltaTable.createIfNotExists(spark) \
    .tableName('silver_store_orders') \
    .addColumn('order_number', 'Integer') \
    .addColumn('customer_id', 'Integer') \
    .addColumn('product_id', 'Integer') \
    .addColumn('order_date', 'Date') \
    .addColumn('units', 'Integer') \
    .addColumn('sale_price', 'Float') \
    .addColumn('currency', 'String') \
    .addColumn('order_mode', 'String') \
    .addColumn('sale_price_usd', 'Float') \
    .addColumn('updated_at', 'Timestamp') \
	.execute()


#Create PRODUCTS silver Table
DeltaTable.createIfNotExists(spark) \
    .tableName('silver_products') \
    .addColumn('product_id', 'Integer') \
    .addColumn('product_name', 'String') \
    .addColumn('product_category', 'String') \
    .addColumn('updated_at', 'Timestamp') \
	.execute()


#Create CURRENCY silver Table
DeltaTable.createIfNotExists(spark) \
    .tableName('silver_currency') \
    .addColumn('currency_value', 'Float') \
    .addColumn('currency_name', 'String') \
    .addColumn('updated_at', 'Timestamp') \
	.execute()


#Create GEOLOCATION silver Table
DeltaTable.createIfNotExists(spark) \
    .tableName('silver_geolocation') \
    .addColumn('ip1', 'Integer') \
    .addColumn('ip2', 'Integer') \
    .addColumn('country_code', 'String') \
    .addColumn('country_name', 'String') \
    .addColumn('updated_at', 'Timestamp') \
	.execute()


#Create LOGS silver Table
DeltaTable.createIfNotExists(spark) \
    .tableName('silver_logs') \
    .addColumn('time', 'String') \
    .addColumn('remote_ip', 'String') \
    .addColumn('country_name', 'String') \
    .addColumn('ip_number', 'Integer') \
    .addColumn('request', 'String') \
    .addColumn('response', 'String') \
    .addColumn('agent', 'String') \
    .addColumn('updated_at', 'Timestamp') \
	.execute()


#Create ECOMM silver Table
DeltaTable.createIfNotExists(spark) \
    .tableName('silver_esalesns') \
    .addColumn('customer_name', 'String') \
    .addColumn('address', 'String') \
    .addColumn('city', 'String') \
    .addColumn('country', 'String') \
    .addColumn('currency', 'String') \
    .addColumn('email', 'String') \
    .addColumn('order_date', 'Date') \
    .addColumn('order_mode', 'String') \
    .addColumn('order_number', 'Integer') \
    .addColumn('phone', 'String') \
    .addColumn('postalcode', 'String') \
    .addColumn('product_name', 'String') \
    .addColumn('sale_price', 'Float') \
    .addColumn('sale_price_usd', 'Float') \
    .addColumn('updated_at', 'Timestamp') \
	.execute()


StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 201, Finished, Available)

**Register the PySpark UDFs**

In [164]:
def mask_value(col):
  column = str(col)
  mask_value = hashlib.sha256(column.encode()).hexdigest()
  return mask_value

def curate_email(email):
  curated_value = email.lower()
  return curated_value

def curate_country(country):
  if (country == 'USA' or country == 'United States'):
    curated_value = 'USA'
  elif (country == 'UK' or country == 'United Kingdom'):
    curated_value = 'UK'
  elif (country == 'CAN' or country == 'Canada'):
    curated_value = 'CAN'
  elif (country == 'IND' or country == 'India'):
    curated_value = 'IND'
  else:
    curated_value = country
  return curated_value

def curate_sales_price(currency, currency_value, sales_price):
  if (currency != 'USD'):
    curated_value = float(sales_price)/float(currency_value)
    return float(curated_value)
  else:
    return float(sales_price)

def ip_to_country(ip):
  ipsplit = ip.split(".")
  ip_number=16777216*int(ipsplit[0]) + 65536*int(ipsplit[1]) + 256*int(ipsplit[2]) + int(ipsplit[3])  
  return ip_number

mask_udf = udf(mask_value, StringType())
curate_email_udf = udf(curate_email, StringType())
curate_country_udf = udf(curate_country, StringType())
curate_sales_price_udf = udf(curate_sales_price, FloatType())
ip_to_country_udf = udf(ip_to_country, StringType())

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 202, Finished, Available)

**Process conversion rates form currency exchange API**

In [165]:
# Create Python function to read data from API

def read_api(url: str):
    normalized_data = dict()
    data = requests.get(api_url).json() 
    normalized_data["_data"] = data # Normalize payload to handle array situtations
    return json.dumps(normalized_data)

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 203, Finished, Available)

In [166]:
payload = json.loads(read_api(api_url))
result = payload.get('_data')
rates = result.get('conversion_rates')
rates["USD"] = 1.00

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 204, Finished, Available)

In [167]:
ratesList = list(rates.items())
rates_df = spark.createDataFrame(ratesList,schema=currency_api_schema)
rates_df = rates_df.withColumn('updated_at', f.lit(const_timestamp))
rates_df.createOrReplaceTempView("currency_src")

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 205, Finished, Available)

In [168]:
%%sql
SELECT * FROM currency_src;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 206, Finished, Available)

<Spark SQL result set with 162 rows and 3 fields>

In [169]:
%%sql
SELECT * FROM silver_currency;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 207, Finished, Available)

<Spark SQL result set with 162 rows and 3 fields>

In [170]:
%%sql

MERGE INTO silver_currency c
USING currency_src u
ON c.currency_name  = u.currency_name 
WHEN MATCHED THEN
  UPDATE SET c.currency_value = u.currency_value, 
             c.updated_at = u.updated_at
WHEN NOT MATCHED THEN 
  INSERT *

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 208, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

In [171]:
%%sql
SELECT * FROM silver_currency;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 209, Finished, Available)

<Spark SQL result set with 162 rows and 3 fields>

**Curate Customers data**

In [172]:
cust_df = spark.sql("SELECT * FROM trg_lakehouse.bronze_store_customers")
curated_cust_df = cust_df.withColumn('email_curated',curate_email_udf('email')).drop('email').withColumnRenamed('email_curated', 'email')

curated_cust_df = curated_cust_df.withColumn('country_curated',curate_country_udf('country')).drop('country').withColumnRenamed('country_curated', 'country')
 
curated_cust_df = curated_cust_df.withColumn('phone_masked',mask_udf('phone')).drop('phone').withColumnRenamed('phone_masked', 'phone')
curated_cust_df = curated_cust_df.withColumn('credit_card_masked',mask_udf('credit_card')).drop('credit_card').withColumnRenamed('credit_card_masked', 'credit_card')
curated_cust_df = curated_cust_df.withColumn('credit_card_masked',mask_udf('credit_card')).drop('credit_card').withColumnRenamed('credit_card_masked', 'credit_card')
curated_cust_df = curated_cust_df.withColumn('address_masked',mask_udf('address')).drop('address').withColumnRenamed('address_masked', 'address')
curated_cust_df = curated_cust_df.withColumn('updated_at', f.lit(const_timestamp))
curated_cust_df.createOrReplaceTempView("store_customers")

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 210, Finished, Available)

In [173]:
%%sql
CREATE OR REPLACE TEMP VIEW store_customers_src AS 
SELECT DISTINCT * FROM store_customers;

SELECT * FROM store_customers_src;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 212, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1000 rows and 10 fields>

In [174]:
%%sql

MERGE INTO silver_store_customers c
USING store_customers_src u
ON c.customer_id = u.customer_id 
WHEN MATCHED THEN
  UPDATE SET c.customer_id = u.customer_id, 
             c.customer_name = u.customer_name,
             c.address = u.address,
             c.city = u.city,
             c.postalcode = u.postalcode,
             c.country = u.country,
             c.phone = u.phone,
             c.email = u.email,
             c.credit_card = u.credit_card,
             c.updated_at = u.updated_at
WHEN NOT MATCHED THEN 
  INSERT *

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 213, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

In [175]:
%%sql
SELECT * FROM silver_store_customers;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 214, Finished, Available)

<Spark SQL result set with 1000 rows and 10 fields>

**Curate Orders data**

In [176]:
order_df = spark.sql("SELECT * FROM trg_lakehouse.bronze_store_orders")
usd_df = spark.sql("""SELECT currency_name AS currency, currency_value 
                        FROM currency_src 
                       WHERE currency_name = 'USD'""")
     
curated_order_df = order_df.join(usd_df, on=['currency'], how="inner")
curated_order_df = curated_order_df.withColumn('sale_price_usd',curate_sales_price_udf('currency', 'currency_value', 'sale_price'))
curated_order_df = curated_order_df.withColumn('updated_at', f.lit(const_timestamp))
curated_order_df = curated_order_df.withColumn('order_date_new', f.to_date(curated_order_df.order_date, 'MM/dd/yyyy')).drop('order_date').withColumnRenamed('order_date_new', 'order_date')
curated_order_df = curated_order_df.drop('currency_value')
curated_order_df.createOrReplaceTempView("store_orders")

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 215, Finished, Available)

In [177]:
%%sql
CREATE OR REPLACE TEMP VIEW store_orders_src AS 
SELECT DISTINCT * FROM store_orders;

SELECT * FROM store_orders_src;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 217, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 669 rows and 10 fields>

In [178]:
%%sql
SELECT * FROM silver_store_orders;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 218, Finished, Available)

<Spark SQL result set with 669 rows and 10 fields>

In [179]:
%%sql

MERGE INTO silver_store_orders c
USING store_orders_src u
ON c.order_number  = u.order_number  
WHEN MATCHED THEN
  UPDATE SET c.order_number = u.order_number, 
             c.customer_id = u.customer_id,
             c.product_id = u.product_id,
             c.order_date = u.order_date,
             c.units = u.units,
             c.sale_price = u.sale_price,
             c.sale_price_usd = u.sale_price_usd,
             c.currency = u.currency,
             c.order_mode = u.order_mode,
             c.updated_at = u.updated_at
WHEN NOT MATCHED THEN 
  INSERT *

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 219, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

In [180]:
%%sql
SELECT * FROM silver_store_orders;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 220, Finished, Available)

<Spark SQL result set with 669 rows and 10 fields>

**Curate Products data**

In [181]:
products_df = spark.sql("SELECT * FROM trg_lakehouse.bronze_products")
curated_products_df = products_df.withColumn('updated_at', f.lit(const_timestamp))
curated_products_df = curated_products_df.withColumnRenamed('product_code','product_id')
curated_products_df.createOrReplaceTempView("products")

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 221, Finished, Available)

In [182]:
%%sql
CREATE OR REPLACE TEMP VIEW products_src AS 
SELECT DISTINCT * FROM products;

SELECT * FROM products_src;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 223, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 10 rows and 4 fields>

In [183]:
%%sql
SELECT * FROM silver_products;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 224, Finished, Available)

<Spark SQL result set with 10 rows and 4 fields>

In [184]:
%%sql

MERGE INTO silver_products c
USING products_src u
ON c.product_id  = u.product_id
WHEN MATCHED THEN
  UPDATE SET c.product_id = u.product_id, 
             c.product_name = u.product_name,
             c.product_category = u.product_category,
             c.updated_at = u.updated_at
WHEN NOT MATCHED THEN 
  INSERT *

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 225, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

In [185]:
%%sql
SELECT * FROM silver_products;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 226, Finished, Available)

<Spark SQL result set with 10 rows and 4 fields>

**Curate Geo Location data**

In [186]:
geolocation_df = spark.sql("SELECT * FROM trg_lakehouse.bronze_geolocation")
curated_geolocation_df = geolocation_df.withColumn('updated_at', f.lit(const_timestamp))
curated_geolocation_df.createOrReplaceTempView("geolocation")

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 227, Finished, Available)

In [187]:
%%sql
CREATE OR REPLACE TEMP VIEW geolocation_src AS 
SELECT DISTINCT * FROM geolocation;

SELECT * FROM geolocation_src;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 229, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1000 rows and 5 fields>

In [188]:
%%sql
SELECT * FROM silver_geolocation;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 230, Finished, Available)

<Spark SQL result set with 1000 rows and 5 fields>

In [189]:
%%sql

MERGE INTO silver_geolocation c
USING geolocation_src u
ON c.ip1  = u.ip1
WHEN MATCHED THEN
  UPDATE SET c.ip1 = u.ip1, 
             c.ip2 = u.ip2,
             c.country_code = u.country_code,
             c.country_name = u.country_name,
             c.updated_at = u.updated_at
WHEN NOT MATCHED THEN 
  INSERT *

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 231, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

In [190]:
%%sql
SELECT * FROM silver_geolocation;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 232, Finished, Available)

<Spark SQL result set with 1000 rows and 5 fields>

**Process E-Com data from kusto**

In [191]:
#This is an example of Reading data from Kusto. Replace T with your <tablename>
kustoQuery = "ecom_orders"
# The Kusto cluster uri to read the data. The query Uri is of the form https://<>.kusto.data.microsoft.com 
kustoUri = "https://trd-5nfr2sdnwwey2uu5rr.z4.kusto.fabric.microsoft.com"
# The database to read the data
database = "ecom"
# The access credentials for the write
accessToken = mssparkutils.credentials.getToken(kustoUri)
kustoDf  = spark.read\
            .format("com.microsoft.kusto.spark.synapse.datasource")\
            .option("accessToken", accessToken)\
            .option("kustoCluster", kustoUri)\
            .option("kustoDatabase", database) \
            .option("kustoQuery", kustoQuery).load()

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 233, Finished, Available)

In [192]:
curated_ecom_df = kustoDf.withColumn('updated_at', f.lit(const_timestamp))
curated_ecom_df = curated_ecom_df.withColumn('phone_masked',mask_udf('phone')).drop('phone').withColumnRenamed('phone_masked', 'phone')
curated_ecom_df = curated_ecom_df.withColumn('address_masked',mask_udf('address')).drop('address').withColumnRenamed('address_masked', 'address')
curated_ecom_df = curated_ecom_df.withColumn('order_date', from_unixtime(unix_timestamp('order_date', 'dd/MM/yyy')))
curated_ecom_df = curated_ecom_df.withColumn('country_curated',curate_country_udf('country')).drop('country').withColumnRenamed('country_curated', 'country')
  
curated_ecom_df = curated_ecom_df.join(usd_df, on=['currency'], how="inner")
curated_ecom_df = curated_ecom_df.withColumn('sale_price_usd',curate_sales_price_udf('currency', 'currency_value', 'sale_price'))

curated_ecom_df = curated_ecom_df.withColumn('order_date_new', to_date(curated_ecom_df.order_date, 'yyyy-MM-dd HH:mm:ss')).drop('order_date').withColumnRenamed('order_date_new', 'order_date')

curated_ecom_df.createOrReplaceTempView("esalesns")

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 234, Finished, Available)

In [193]:
%%sql
CREATE OR REPLACE TEMP VIEW esalesns_src AS 
SELECT DISTINCT * FROM esalesns;

SELECT * FROM esalesns_src;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 236, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 26 rows and 16 fields>

In [194]:
%%sql
SELECT * FROM silver_esalesns;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 237, Finished, Available)

<Spark SQL result set with 26 rows and 15 fields>

In [195]:
%%sql

MERGE INTO silver_esalesns c
USING esalesns_src u
ON c.email  = u.email
WHEN MATCHED THEN
  UPDATE SET c.customer_name = u.customer_name, 
             c.address = u.address,
             c.city = u.city,
             c.country = u.country,
             c.currency = u.currency,
             c.email = u.email,
             c.order_date = u.order_date,
             c.order_mode = u.order_mode,
             c.order_number = u.order_number,
             c.phone = u.phone,
             c.postalcode = u.postalcode,
             c.product_name = u.product_name,
             c.sale_price = u.sale_price,
             c.sale_price_usd = u.sale_price_usd,
             c.updated_at = u.updated_at
WHEN NOT MATCHED THEN 
  INSERT *

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 238, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

In [196]:
%%sql
SELECT * FROM silver_esalesns;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 239, Finished, Available)

<Spark SQL result set with 26 rows and 15 fields>

**Get the logs data from git and Curate**

In [197]:
for file in log_file_list:
    data = pd.read_json(file, lines=True) 
    pd_dfs.append(data) 

pd_df = pd.concat(pd_dfs, ignore_index=True)

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 240, Finished, Available)

In [198]:
logs_df = spark.createDataFrame(pd_df)
curated_logs_df = logs_df.withColumn('updated_at', f.lit(const_timestamp))
curated_logs_df = curated_logs_df.withColumn('time', from_unixtime(unix_timestamp('time', 'dd/MM/yyy:HH:m:ss')))
curated_logs_df = curated_logs_df.withColumn('ip_number',ip_to_country_udf('remote_ip'))
curated_logs_df = curated_logs_df.withColumn("ip_number_int", curated_logs_df['ip_number'].cast('int')).drop('ip_number').withColumnRenamed('ip_number_int', 'ip_number')
curated_logs_df.createOrReplaceTempView('logs')

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 241, Finished, Available)

In [199]:
%%sql
CREATE OR REPLACE TEMP VIEW logs_src AS 
SELECT DISTINCT * FROM
(SELECT logs.time
	,logs.remote_ip
	,geolocation_src.country_name
	,logs.ip_number
	,logs.request
	,logs.response
	,logs.agent
	,logs.updated_at
FROM logs
JOIN geolocation_src
WHERE ip1 <= ip_number
	AND ip2 >= ip_number);

SELECT * FROM logs_src;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 243, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1000 rows and 8 fields>

In [200]:
%%sql
SELECT * FROM silver_logs;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 244, Finished, Available)

<Spark SQL result set with 1000 rows and 8 fields>

In [201]:
%%sql

MERGE INTO silver_logs c
USING logs_src u
ON c.remote_ip   = u.remote_ip 
WHEN NOT MATCHED THEN 
  INSERT *

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 245, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

In [202]:
%%sql
SELECT * FROM silver_logs;

StatementMeta(, 65629300-5f45-4047-a9a1-fce395796792, 246, Finished, Available)

<Spark SQL result set with 1000 rows and 8 fields>